In [44]:
import re
import os

In [45]:
# Read file

In [46]:
with open("Standard2.svg") as f:
    filecontent = f.read().split("<defs/>\n ")

In [47]:
len(filecontent)

94

In [48]:
# Find numbers and sort SVG lines

In [49]:
svgDict = {}

In [59]:
for idx, single_svg in enumerate(filecontent):
    if idx != 0:
        match = re.search(r'fid_(\d+)', single_svg)
        
        lines = single_svg.split("\n")
        newLines = []
        for idx, line in enumerate(lines):
            if line.strip().startswith("<g inkscape:groupmode=\"layer\""):
                newLines.append(line)
            elif line.strip().startswith("<g fill-opacity=\"1\""):
                l = re.sub(r'transform="[^"]*"', '', line)
                newLines.append(l)
            elif line.strip().startswith("<path d="):
                newLines.append(line)
        newLines.append("\n</g>\n</g>")
                
        
        svgDict[int(match.group(1))] = "\n".join(newLines)

In [60]:
extremePoints = {}

In [61]:
svgDict[1]

'<g inkscape:groupmode="layer" stroke-width="1" stroke-linejoin="bevel" stroke-linecap="square" inkscape:label="Map 1: fid_1" fill="none" stroke="black" id="Map 1: fid_1" fill-rule="evenodd">\n  <g fill-opacity="1" stroke-width="2" stroke-linejoin="bevel" font-weight="400" stroke-linecap="square" font-size="83.3333" font-style="normal" fill="#8d5a99" stroke-opacity="1"  font-family="Noto Sans" stroke="#232323">\n   <path d="M3472.71,3180.42 L3542.43,3190.34 L3538.74,3142.73 L3546.11,3141.88 L3542.71,3121.76 L3550.65,3115.52 L3542.43,3086.62 L3548.38,3072.44 L3559.15,3057.14 L3554.05,3038.44 L3532.51,3005.84 L3521.46,3020.3 L3507,3027.67 L3506.44,2989.97 L3478.09,2935.27 L3467.04,2934.14 L3461.94,2946.89 L3463.64,2960.21 L3459.11,2961.35 L3407.81,2927.62 L3394.2,2948.03 L3386.55,2942.93 L3372.1,2965.03 L3407.52,2985.15 L3433.03,3014.91 L3472.71,3180.42" vector-effect="none" fill-rule="evenodd"/>\n\n</g>\n</g>'

In [23]:
svgDict[1]

'<g inkscape:groupmode="layer" stroke-width="1" stroke-linejoin="bevel" stroke-linecap="square" inkscape:label="Map 1: fid_1" fill="none" stroke="black" id="Map 1: fid_1" fill-rule="evenodd">\n  <g stroke-width="1" stroke-linejoin="bevel" font-weight="400" stroke-linecap="square" font-size="83.3333" font-style="normal" fill="none" stroke-opacity="1" transform="matrix(1,0,0,1,0,0)" font-family="Noto Sans" stroke="#000000"/>\n  <g stroke-width="1" stroke-linejoin="bevel" font-weight="400" stroke-linecap="square" font-size="83.3333" font-style="normal" fill="none" stroke-opacity="1" transform="matrix(1,0,0,1,0,0)" font-family="Noto Sans" stroke="#000000"/>\n  <g stroke-width="1" stroke-linejoin="bevel" font-weight="400" stroke-linecap="square" font-size="83.3333" font-style="normal" fill="none" stroke-opacity="1" transform="matrix(23.619,0,0,23.619,-728.486,-229.753)" font-family="Noto Sans" stroke="#000000"/>\n  <g stroke-width="1" stroke-linejoin="bevel" font-weight="400" stroke-linecap

In [72]:
def findExtremePoints(pathstr):
    arr = pathstr.split()
    min_x = float("inf")
    min_y = float("inf")
    max_x = float("-inf")
    max_y = float("-inf")
    for a in arr:
        x,y = a.replace("L","").replace("M","").split(",")
        x = float(x)
        y = float(y)
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
    #return f"{round(float(min_x+20), 3)} {round(float(min_y+20), 3)} {round(float(max_x-min_x+40), 3)} {round(float(max_y-min_y+40), 3)}"
    return f"{round(float(min_x-5), 3)} {round(float(min_y-5), 3)} {round(float(max_x-min_x+10), 3)} {round(float(max_y-min_y+10), 3)}"
    

In [73]:
# Change size of shape

In [74]:
for idx, line in svgDict.items():
    if idx != 0:
        #cont = svgDict[idx].split("\n")
        #match = re.search(r'path', line)
        pattern = r'<path[^>]*\s+d="([^"]+)"'
        
        matches = re.findall(pattern, line)
        
        extremePoints[idx] = findExtremePoints(matches[0])

In [75]:
extremePoints

{59: '3369.75 2587.24 50.05 53.71',
 70: '3072.5 1724.32 280.39 186.32',
 69: '2996.02 1436.22 251.12 338.13',
 68: '3015.42 1245.8 398.56 290.93',
 67: '3612.71 1664.2 227.23 230.42',
 66: '3417.07 1657.19 224.48 255.28',
 65: '3183.52 1512.5 319.73 339.03',
 64: '3044.7 1059.1 139.28 197.69',
 63: '3252.6 2625.09 93.22 76.82',
 62: '3279.9 2666.58 73.65 76.12',
 61: '3322.83 2665.75 95.32 76.27',
 60: '3298.08 2708.67 81.25 71.47',
 71: '3360.99 1985.67 161.97 160.33',
 58: '3400.34 2618.49 50.54 73.12',
 57: '3309.38 2603.86 103.75 71.02',
 56: '3377.31 2640.53 70.57 77.15',
 55: '3405.52 2697.89 62.92 67.28',
 54: '3366.37 2712.12 89.4 100.01',
 53: '3339.19 2734.65 65.8 64.77',
 52: '3423.45 2617.67 66.74 72.13',
 51: '3391.37 2561.75 111.16 86.32',
 50: '3448.24 2674.46 110.66 131.01',
 49: '3424.89 2669.77 110.2 146.35',
 48: '3436.56 2745.8 92.9 72.42',
 82: '3368.09 3802.31 200.15 298.84',
 93: '3100.71 4010.14 304.44 385.07',
 92: '3379.69 3926.33 344.57 366.08',
 91: '3823.4

In [76]:
# Write SVGs

In [77]:
for idx in range(1,len(filecontent)):
    filename = f"../public/images/countries/{idx}/vector.svg"
    #os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(f"{filename}","w") as toWrite:
        toWrite.write(f"<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.2\" baseProfile=\"tiny\" xmlns:inkscape=\"http://www.inkscape.org/namespaces/inkscape\" width=\"315.045mm\" viewBox=\"{extremePoints[idx]}\" height=\"227.076mm\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" xmlns:cc=\"http://creativecommons.org/ns#\">\n")
        toWrite.write(f"{svgDict[idx]}\n")
        toWrite.write("</svg>")